In [2]:
pip install wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.8/313.8 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
import wandb
from wandb.integration.keras import WandbMetricsLogger
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Define paths for the dataset
train_dir = '/content/drive/MyDrive/Sri/Lung/Lung Disease Dataset/train/'
test_dir = '/content/drive/MyDrive/Sri/Lung/Lung Disease Dataset/test/'
validation_dir = '/content/drive/MyDrive/Sri/Lung/Lung Disease Dataset/val/'

In [ ]:
wandb.init(project="Lung_Disease", config={
    "epochs": 25,
    "batch_size": 32,
    "learning_rate": 0.0001,
    "l2_reg": 0.001,
    "architecture": "CNN"
})
config = wandb.config


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bala46308 (bala46308-chettinadclg). Use `wandb login --relogin` to force relogin


In [ ]:
# Constants
IMG_HEIGHT, IMG_WIDTH = 150, 150
BATCH_SIZE = 32

In [ ]:
# Function to load images and apply data augmentation
def load_data(data_dir, batch_size):
    # Load images
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=batch_size,
        label_mode='categorical'
    )

    # Data augmentation
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomZoom(0.1),
    ])

    return dataset.map(lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
# Load training and validation data
train_dataset = load_data(train_dir, BATCH_SIZE)

Found 6054 files belonging to 5 classes.


In [ ]:
validation_dataset = load_data(validation_dir, BATCH_SIZE)

Found 2016 files belonging to 5 classes.


In [ ]:
test_dataset = load_data(test_dir, BATCH_SIZE)

Found 2025 files belonging to 5 classes.


In [ ]:
# Build the CNN model with additional layers
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, (3, 3), activation='relu'),  # New layer
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(512, (3, 3), activation='relu'),  # New layer
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(256, activation='relu'),  # Increased number of units
    Dropout(0.5),
    Dense(5, activation='softmax')  # 5 classes
])


In [ ]:
# Compile the model with a learning rate of 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Early stopping to stop training when validation loss doesn't improve
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)



In [ ]:
# Train the model
history = model.fit(
    train_dataset,
    epochs=25,
    validation_data=validation_dataset,
    callbacks=[WandbMetricsLogger(), early_stopping]
)


Epoch 1/25
190/190 ━━━━━━━━━━━━━━━━━━━━ 123s 621ms/step - accuracy: 0.3135 - loss: 4.1606 - val_accuracy: 0.5660 - val_loss: 1.1395
Epoch 2/25
190/190 ━━━━━━━━━━━━━━━━━━━━ 117s 508ms/step - accuracy: 0.5239 - loss: 1.1838 - val_accuracy: 0.5541 - val_loss: 1.0569
Epoch 3/25
190/190 ━━━━━━━━━━━━━━━━━━━━ 163s 619ms/step - accuracy: 0.5557 - loss: 1.0549 - val_accuracy: 0.5928 - val_loss: 0.9960
Epoch 4/25
190/190 ━━━━━━━━━━━━━━━━━━━━ 125s 529ms/step - accuracy: 0.5789 - loss: 0.9899 - val_accuracy: 0.6369 - val_loss: 0.8369
Epoch 5/25
190/190 ━━━━━━━━━━━━━━━━━━━━ 100s 519ms/step - accuracy: 0.6109 - loss: 0.9267 - val_accuracy: 0.6508 - val_loss: 0.8507
Epoch 6/25
190/190 ━━━━━━━━━━━━━━━━━━━━ 116s 610ms/step - accuracy: 0.6251 - loss: 0.8888 - val_accuracy: 0.6523 - val_loss: 0.8301
Epoch 7/25
190/190 ━━━━━━━━━━━━━━━━━━━━ 100s 526ms/step - accuracy: 0.6412 - loss: 0.8557 - val_accuracy: 0.6761 - val_loss: 0.8007
Epoch 8/25
190/190 ━━━━━━━━━━━━━━━━━━━━ 116s 606ms/step - accuracy: 0.6531 -

In [ ]:
# Save the model
model.save('lung_disease_classifier_with_early_stopping.keras')


In [ ]:
# Plot training & validation accuracy/loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))


In [ ]:
# Accuracy plot
plt.figure(figsize=(8, 6))
plt.plot(epochs, acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

In [ ]:
# Loss plot
plt.figure(figsize=(8, 6))
plt.plot(epochs, loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()